# CAMBIO

In [37]:
from cambio import CambioIo
from scipy import stats
import numpy as np

In [ ]:
input_file = input() ## enter path to CAMBio_Tabelle

In [7]:
import pandas as pd

In [11]:
df = pd.read_csv("../../../Documents/manuscripts/AMD/2025_07_24_CAMBio_Tabelle_für_Hannes_Urbat.csv", sep=";")
df

,Alias,Eye,Status,Healthy Retina,No AMD,Group,Early AMD,Dry AMD,Geographic Atrophy,original Aqueous Humor Volume (µL),...,C__2530382_10,Datenüberürfung,Eye.1,Status.1,Healthy Retina.1,No AMD.1,Group.1,Early AMD.1,Dry AMD.1,Geographic Atrophy.1
0,1,L,Completed,Yes,Yes,Control,No,No,No,90,...,G,NaN,L,Completed,Yes,Yes,Control,Yes,Yes,Yes
1,1,R,Completed,Yes,Yes,Control,No,No,No,65,...,G,NaN,R,Lost,No,No,AMD,No,No,No
2,2,L,Completed,Yes,Yes,Control,No,No,No,40,...,AG,NaN,NaN,Excluded,NaN,NaN,NaN,NaN,NaN,NaN
3,2,R,Completed,Yes,Yes,Control,No,No,No,40,...,AG,NaN,NaN,Pending,NaN,NaN,NaN,NaN,NaN,NaN
4,3,L,Completed,Yes,Yes,Control,No,No,No,200,...,AG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,100,R,Completed,No,No,AMD,No,Yes,Yes,180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,101,L,Pending,No,Yes,Control,No,No,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,101,R,Pending,No,Yes,Control,No,No,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,102,L,Pending,Yes,Yes,Control,No,No,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
!ls ../../../Documents/manuscripts/AMD/2025_07_24_CAMBio_Tabelle_für_Hannes_Urbat.csv

../../../Documents/manuscripts/AMD/2025_07_24_CAMBio_Tabelle_für_Hannes_Urbat.csv


In [14]:
i = 0
for c in df.columns:
    print(f"{i}) {c}")
    i += 1

0) Alias
1) Eye
2) Status
3) Healthy Retina
4) No AMD
5) Group
6) Early AMD
7) Dry AMD
8) Geographic Atrophy
9) original Aqueous Humor Volume (µL)
10) arrived and freezed
11) measured in NanoFCM
12) storage duration until 1st measurement [days]
13) EV concentration (NanoFCM (part./mL))
14) EV concentration (NanoFCM (part./mL)) 2nd measurement 04.03.2025
15) EV median size ((nm), NanoFCM)
16) EV median size ((nm), NanoFCM) 2nd measurement 04.03.2025
17) Fluorescent Particle Count total CD41a (ExoView)
18) Fluorescent Particle Count total CD63 (ExoView)
19) Fluorescent Particle Count total CD81 (ExoView)
20) Fluorescent Particle Count total CD9 (ExoView)
21) Fluorescent Particle Count FI CD41a (ExoView)
22) Fluorescent Particle Count FI CD63 (ExoView)
23) Fluorescent Particle Count FI CD81 (ExoView)
24) Fluorescent Particle Count FI CD9 (ExoView)
25) Fluorescent Particle Count FH CD41a (ExoView)
26) Fluorescent Particle Count FH CD63 (ExoView)
27) Fluorescent Particle Count FH CD81 (ExoV

In [19]:
df[["Alias", "Group", "original Aqueous Humor Volume (µL)"]]

,Alias,Group,original Aqueous Humor Volume (µL)
0,1,Control,90
1,1,Control,65
2,2,Control,40
3,2,Control,40
4,3,Control,200
...,...,...,...
159,100,AMD,180
160,101,Control,NaN
161,101,Control,NaN
162,102,Control,NaN


In [23]:
groups = df.groupby("Group")["original Aqueous Humor Volume (µL)"].apply(list)

groups

Group
AMD                 [130, 80, nan, nan, 65, 50, 35, 160, 140, 100,...
Control             [90, 65, 40, 40, 200, 160, 35, 120, 200, 50, 5...
Control (Gliose)                                            [70, nan]
Name: original Aqueous Humor Volume (µL), dtype: object

In [40]:
col = "original Aqueous Humor Volume (µL)"

s = df[col].astype(str).str.strip()
# mark strings containing any letters as invalid
mask_alpha = s.str.contains(r'[A-Za-z]', na=False)
s_clean = s.copy()
s_clean[mask_alpha] = np.nan

# now coerce remaining to numeric
df[col] = pd.to_numeric(s_clean, errors="coerce")
#for row, index in groups.iterrows()
groups = (
    df[df["Group"] != "Gliose"]  # exclude Gliose rows
    .groupby("Group")["original Aqueous Humor Volume (µL)"]
    .apply(list)
)
g1, g2 = groups.iloc[0], groups.iloc[1]


In [41]:


t_stat, p_val = stats.ttest_ind(g1, g2, equal_var=False,  nan_policy='omit')  # Welch's t-test
print(f"T-statistic: {t_stat:.4f}, p-value: {p_val:.4g}")

T-statistic: -1.7917, p-value: 0.07534
